In [172]:
import pandas as pd
import climdist.data
import json
import spacy
from spacy import displacy
from tqdm import tqdm
import random
from climdist.doccano import Transformer
import numpy as np
import re

In [28]:
df = 0

In [3]:
df.head()

,date,year,month,day,pub,heading,full_text,href,text_len,readability,heading2
0,1802-01-01,1802,1,1,Rigasche Zeitung,"St. Petersburg, den 20. Decemb.","St. Petersburg, den 20. Decemb.\n\n\tAuf Aller...",https://proc.dom.lndb.lv/file.axd?id=3377815&a...,2067,1,St. Petersburg
1,1802-01-01,1802,1,1,Rigasche Zeitung,"Paris, den 18ten December.","Paris, den 18ten December.\n\n\tDer Oberconsu!...",https://proc.dom.lndb.lv/file.axd?id=3377818&a...,3664,1,Paris
2,1802-01-01,1802,1,1,Rigasche Zeitung,"Haag, den 22sten December.","Haag, den 22sten December.\n\n\tIn kurzem erwa...",https://proc.dom.lndb.lv/file.axd?id=3377819&a...,967,1,Haag
3,1802-01-01,1802,1,1,Rigasche Zeitung,"Zürich, den 16ten December.","Zürich, den 16ten December.\n\n\tDie durch die...",https://proc.dom.lndb.lv/file.axd?id=3377822&a...,482,1,Zürich
4,1802-01-01,1802,1,1,Rigasche Zeitung,"Wien, den 19ten December.","Wien, den 19ten December.\n\n\tDie Malcheser-N...",https://proc.dom.lndb.lv/file.axd?id=3377823&a...,1154,1,Wien


In [36]:
precip = ['Regen',
          'Regens',
          'Regenfall',
          'Niederschlag',
          'Niederschlages',
          'Guss',
          'Regenschauer',
          'Regenguss',
          'Wolkenbruch',
          'Regenguß',
          'Schnee',
          'Schneefall',
          'Niederschlag',
          'Schneebruch',
          'Dürre',
          'Dürres',
          'Graupel',
          'Hagel',
          'Hagels',
          'Hagelschlag',
          'Gewitter',
          'Gewitters',
          'Tau',
          'Thau',
          'Tauwetter',
          'Thauwetter',
          'Nebel',
          'Nebels']

wind = ['Wind',
        'Winde',
        'Windes',
        'Winden',
        'Sturm',
        'Stürme',
        'Stürmen',
        'Sturmes',
        'Sturmwind',
        'Tornado',
        'Orkan',
        'Wirbelwind',
        'Wirbelsturm',
        'Unwetter',
        'Taifun',
        'Südwind',
        'Südwestwind',
        'Westwind',
        'Nordwestwind',
        'Nordwind',
        'Nordostwind',
        'Ostwind',
        'Südostwind',
        'S. Wind',
        'S.W. Wind',
        'S. W. Wind',
        'W. Wind',
        'N.W. Wind',
        'N. W. Wind',
        'N. Wind',
        'N.O. Wind',
        'N. O. Wind',
        'O. Wind',
        'S.O. Wind',
        'S. O. Wind']

flood = ['Flut',
         'Sutrmflut',
         'Hochwasser',
         'Hochflut',
         'Überschwemmung',
         'Überschweummungs',
         'Überschwemmungen',
         'Überflutung',
         'Überflutungen']

temperature = ['Kälte',
               'Kältes',
               'Wärme',
               'Wärmes',
               'Hitze',
               'Hitzewelle',
               'Kältewelle',
               'Frost']

general = ['Wetter',
           'Wetters',
           'Witterung',
           'Witterungs',
           'Klima',
           'Klimas',
           'Klimaereignis',
           'Wetterereignis',
           'Windstärke',
           'Luftdruck']

In [37]:
weather_events = {'general': general,
                  'temperature': temperature,
                  'precipitations': precip,
                  'wind': wind,
                  'flood': flood}

In [127]:
# save the weather events for entity ruler

with open('../pipeline/weather_events.json', 'w', encoding='utf8') as f:
    json.dump(weather_events, f)

In [38]:
# create patterns for ruler (assign label by surface text)

patterns = []

for category in weather_events.values():
    for phrase in category:
        if ' ' in phrase:
            patterns.append({
                'label': 'WEA',
                'pattern': [{'LOWER': piece} for piece in phrase.split(' ')]
            })
        else:
            patterns.append({'label': 'WEA', 'pattern': phrase})

In [39]:
patterns

[{'label': 'WEA', 'pattern': 'Wetter'},
 {'label': 'WEA', 'pattern': 'Wetters'},
 {'label': 'WEA', 'pattern': 'Witterung'},
 {'label': 'WEA', 'pattern': 'Witterungs'},
 {'label': 'WEA', 'pattern': 'Klima'},
 {'label': 'WEA', 'pattern': 'Klimas'},
 {'label': 'WEA', 'pattern': 'Klimaereignis'},
 {'label': 'WEA', 'pattern': 'Wetterereignis'},
 {'label': 'WEA', 'pattern': 'Windstärke'},
 {'label': 'WEA', 'pattern': 'Luftdruck'},
 {'label': 'WEA', 'pattern': 'Kälte'},
 {'label': 'WEA', 'pattern': 'Kältes'},
 {'label': 'WEA', 'pattern': 'Wärme'},
 {'label': 'WEA', 'pattern': 'Wärmes'},
 {'label': 'WEA', 'pattern': 'Hitze'},
 {'label': 'WEA', 'pattern': 'Hitzewelle'},
 {'label': 'WEA', 'pattern': 'Kältewelle'},
 {'label': 'WEA', 'pattern': 'Frost'},
 {'label': 'WEA', 'pattern': 'Regen'},
 {'label': 'WEA', 'pattern': 'Regens'},
 {'label': 'WEA', 'pattern': 'Regenfall'},
 {'label': 'WEA', 'pattern': 'Niederschlag'},
 {'label': 'WEA', 'pattern': 'Niederschlages'},
 {'label': 'WEA', 'pattern': 'G

In [48]:
nlp = spacy.load('de_core_news_md')
nlp.disable_pipe('ner')
nlp.add_pipe('entity_ruler')
ruler = nlp.get_pipe('entity_ruler')
ruler.add_patterns(patterns)
ruler.to_disk('../data/models/entity_ruler_121021')

## First annotation batch

In [17]:
with open('../pipeline/weather_events.json', 'r', encoding='utf8') as f:
    dico = json.load(f)

In [53]:
# get df

df = climdist.data.load('main', readability=True, heading2=False)

In [51]:
def choose_annotation_data(df, n=100, max_len=5000):
    
    results = []
    
    while len(results) < n:
        
        sample = df[(df.readability==True) & (df.text_len < max_len)].sample(n)
        
        for i in tqdm(sample.index):
            doc = nlp(df.loc[i, 'full_text'])
            if len(doc.ents) > 0:
                results.append(i)
                
        for i in results:
            if results.count(i) > 1:
                results.pop(i)
                
        print(f'{len(results)} results so far')
                
    results = results[:n+1]
                
    print('Finished')
    
    return results

In [118]:
choose_annotation_data(df)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

7 results so far


  2%|█▋                                                                                | 2/100 [00:00<00:05, 18.89it/s]

11 results so far


  3%|██▍                                                                               | 3/100 [00:00<00:03, 29.69it/s]

...

92 results so far


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

99 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.64it/s]

106 results so far
Finished


In [71]:
import json

with open('../pipeline/ner/annotation_batch_index_121021.json') as f:
    ix = json.load(f)

In [72]:
# select the texts corepsonding to indices
annotation_selection = df.iloc[ix]

In [73]:
# grab the old model to assign entities as best as possible before starting annotation
import spacy
nlp = spacy.load('../data/archive/models/spacy_model_250421/')

c:\users\krister\climdist\v_env\lib\site-packages\spacy\util.py:758: UserWarning: [W095] Model 'de_core_news_md' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [74]:
# convert to doccano format
dt = Transformer()
dt.pandas_to_doccano(annotation_selection, nlp, '../pipeline/ner/annotation_batch_131021.jsonl')

File created!


## Second annotation batch

In [54]:
### create second annotation batch (13.11.21)

new_batch_index = choose_annotation_data(df)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.91it/s]


5 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.41it/s]


17 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.61it/s]


...

85 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.47it/s]


97 results so far


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.82it/s]

108 results so far
Finished


In [58]:
# load the old index to check against duplicate entries
with open('../pipeline/ner/annotation_batch_index_121021.json', 'r', encoding='utf8') as f:
    old_index = json.load(f)
    
for ix in new_batch_index:
    if ix in old_index:
        print(ix)

In [60]:
# save the new index
with open('../pipeline/ner/annotation_batch_index_131121.json', 'w', encoding='utf8') as f:
    json.dump(new_batch_index, f)

In [75]:
# create the new selection with the index
new_annotation_selection = df.iloc[new_batch_index]

In [76]:
# now grab the old model trained after the previous batch
nlp = spacy.load('../data/models/main_2/model-best/')

In [77]:
# convert to doccano format
dt = Transformer()
dt.pandas_to_doccano(new_annotation_selection, nlp, '../pipeline/ner/annotation_batch_131121.jsonl')

File created!


## Prepare annotated data for training

In [186]:
annotated_data = []

with open('../pipeline/ner/annotated_data_151121.jsonl', 'r', encoding='utf8') as f:
    for line in list(f):
        annotated_data.append(json.loads(line))

In [196]:
for entry in annotated_data:
    if len(entry['annotations']) == 0:
        print(entry)

In [204]:
dt = Transformer()
TRAIN_DATA = dt.doccano_to_spacy('../pipeline/ner/annotated_data_151121.jsonl')

Konstantinopel
Tschatirdagh.
Specify the label for this type of entity: LOC


Allah Burgen
Specify the label for this type of entity: PER


Sturm
Schnee
Donner
Specify the label for this type of entity: WEA


И- August
vom
30. August
vom 31. Juli
Specify the label for this type of entity: DAT


{24: 'LOC', 28: 'PER', 29: 'WEA', 23: 'DAT'}
Doccano data imported


In [201]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [205]:
cleaned_data = trim_entity_spans(TRAIN_DATA)

In [206]:
TRAIN_DATA == cleaned_data

True

In [208]:
from spacy.tokens import DocBin

def convert_to_spacy3x(TRAIN_DATA, output_path):
    
    nlp = spacy.blank("de") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in TRAIN_DATA: # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print(text[start:end])
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        if len(ents) == 0:
            print(text)
        db.add(doc)

    db.to_disk(output_path) # save the docbin object

In [207]:
# split into training and dev sets (85/15)

np.random.shuffle(TRAIN_DATA) # shuffle the data in-place
train_set = TRAIN_DATA[0:338]
valid_set = TRAIN_DATA[338:]

In [209]:
convert_to_spacy3x(train_set, '../pipeline/ner/train.spacy')
convert_to_spacy3x(valid_set, '../pipeline/ner/valid.spacy')

Lincoln'S
Skipping entity
Wohlershof
Skipping entity
Praaa
Skipping entity
Schnee
Skipping entity
Alexander
Skipping entity
Witterung
Skipping entity
OeselS
Skipping entity
Baden
Skipping entity
W3!W
Skipping entity
Riga
Skipping entity
Mikau
Skipping entity
Warschan
Skipping entity
Coblenz
Skipping entity
Schneegestöber
Skipping entity
Spanien
Skipping entity
Sübfrankretch
Skipping entity
Wilna
Skipping entity
Mailand
Skipping entity
Zrankfurt
Skipping entity
Wisby
Skipping entity
Carysto
Skipping entity
Mitau
Skipping entity
Sturm
Skipping entity
Paris
Skipping entity
Doßingen
Skipping entity
Süd-Karolma
Skipping entity
Vacca
Skipping entity
LübeS
Skipping entity
Skassy
Skipping entity
Diebitsch
Skipping entity
Goethe
Skipping entity


In [180]:
def find_trailing_whitespace(TRAIN_DATA):
    
    for text in TRAIN_DATA:
        for entity in text[1]['entities']:
            entity_surface = text[0][entity[0]:entity[1]]
            if entity_surface != entity_surface.strip() or entity_surface != entity_surface.lstrip():
                print(text[0])
                print('--------',entity_surface, '\n')
                
find_trailing_whitespace(TRAIN_DATA)

In [184]:
def find_whitespace_in_entities(TRAIN_DATA):
    
    whitespace = re.compile(r'\s')
    ct = 0
    
    for text in TRAIN_DATA:
        print(ct)
        for entity in text[1]['entities']:
            entity_surface = text[0][entity[0]:entity[1]]
            if whitespace.match(entity_surface[0]) or whitespace.match(entity_surface[-1]):
                print(text[0])
                print('--------',entity_surface, '\n')
        ct += 1
                
find_whitespace_in_entities(TRAIN_DATA)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


IndexError: string index out of range

In [185]:
TRAIN_DATA[224]

['Lesefrüchte.\n\nKleinmiithige Seelen lassen in, Sturm des Lebens daS Steuer aus den\nHänden fahren und geben ihr Heil den Launen von Wind und Wellen preis,\noder erwarten zu ihrer Rettung den Eingriff einer höheren Macht. Der Mu\nthige steuert mit fester Hand durch die Brandung hindurch , um auf dein Riff\nmit dem Leben zugleich die Kraft zu neuen Kämpfen zu retten. D. B.',
 {'entities': [[0, 18, 'LOC'],
   [1265, 1276, 'PER'],
   [21, 25, 'LOC'],
   [56, 60, 'LOC'],
   [234, 251, 'PER'],
   [1055, 1066, 'LOC'],
   [1225, 1240, 'PER'],
   [1248, 1263, 'PER'],
   [1282, 1292, 'PER'],
   [1314, 1334, 'PER'],
   [1336, 1346, 'PER'],
   [1348, 1355, 'PER'],
   [1360, 1370, 'PER']]}]

In [59]:
def convert_train_data(data):
    
    new_data = []
    
    for old_entry in data:
        new_entry = []
        new_entry.append(old_entry['text'])
        new_entry.append({})
        new_entry[1]['entities'] = old_entry['entities']
        new_data.append(new_entry)
        
    return new_data

In [138]:
#convert_train_data(train_data)

In [117]:
def explore_results():
    i = random.choice(results)
    doc = nlp(df.loc[i, 'full_text'])
    print(df.loc[i, 'text_len'])
    displacy.render(doc, style='ent', jupyter=True)

In [106]:
explore_results()

4152


In [22]:
def print_year(start):
    
    index = start
    
    while True:
        
        current = ix[index]
        print(index)
        print(df.loc[current, 'year'])
        print('\n')
        print(df.loc[current, 'full_text'][:100])
        cont = input('Next:')
        index += 1
        print('\n\n\n')